In [409]:
import pandas as pd
import numpy as np
import os
import re
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from linearmodels.system import SUR
from pathlib import Path
from IPython.display import display
from scipy.stats import pearsonr
from collections import OrderedDict
from sklearn.linear_model import LinearRegression
from linearmodels.system.results import WaldTestStatistic


import warnings
warnings.filterwarnings('ignore')

In [410]:
root = Path(os.getcwd())

In [411]:
def set_timeframe(data, start_date, end_date):
    data = data.loc[data.index.get_level_values('date') >= start_date]
    data = data.loc[data.index.get_level_values('date') <= end_date]
    return data

In [412]:
predictors = pd.read_parquet(root / 'data' / 'predictors.parquet.brotli')

In [413]:
predictors = set_timeframe(predictors, '1925-01-01', '2022-01-01')

# Handling Data

In [414]:
regression = predictors.copy(deep=True)

In [415]:
# regression = regression.dropna(subset = ['two_vwr_ind', 'vol_vw', 'vol_vw_change', 'turnover_vw', 
#                 'turnover_vw_change', 'age_market_rank', 'age_tilt', 'issuance', 'btm_vwr', 
#                 'revenue_market_rank', 'cape', 'acc'])
regression = regression.fillna(0)

In [416]:
# Bubble dates from paper
bubble_date = pd.read_csv(root / 'data' / 'bubbles.csv')
bubble_date = bubble_date.applymap(lambda s: s.lower() if type(s) == str else s)
bubble_date.date = pd.to_datetime(bubble_date.date)
bubble_date.date = bubble_date.date.dt.to_period('M').dt.to_timestamp('M')
bubble_date = bubble_date.set_index(['date', 'ind'])

In [417]:
# Add excess returns
regression['rf_two_vwr_ind'] = regression['two_vwr_ind'] - regression['rf']/100
regression['market_two_vwr_ind'] = regression['two_vwr_ind'] - regression['market_return']

In [418]:
regression.index.get_level_values(1).unique()

Index(['aero', 'agric', 'autos', 'banks', 'beer', 'bldmt', 'books', 'boxes',
       'bussv', 'chems', 'chips', 'clths', 'cnstr', 'coal', 'drugs', 'elceq',
       'fin', 'food', 'fun', 'hardw', 'hshld', 'insur', 'labeq', 'mach',
       'meals', 'medeq', 'mines', 'oil', 'rlest', 'rtail', 'ships', 'smoke',
       'steel', 'telcm', 'trans', 'txtls', 'util', 'whlsl', 'toys', 'persv',
       'other', 'paper', 'rubbr', 'gold', 'guns', 'hlth', 'soda', 'softw',
       'fabpr'],
      dtype='object', name='ind')

# Creating Table 6 (Bubbles for Fama)

In [419]:
def get_post_bubble_data(df, date, ind, months=24):
    start_date = pd.to_datetime(date)
    end_date = start_date + pd.DateOffset(months=months)
    mask = (df.index.get_level_values('date') > start_date) & (df.index.get_level_values('date') <= end_date) & (df.index.get_level_values('ind') == ind)
    return df[mask]

In [420]:
def set_correct_format(results):
    # 2. Extract series data into separate columns for 'b'
    df = pd.DataFrame(results)
    # Extract the coefficients ('b') and t-values ('t')
    df_b = df['b'].apply(pd.Series)
    df_t = df['t'].apply(pd.Series)
    # Rename columns to indicate they contain coefficients or t-values
    df_b.columns = [f"b_{col}" for col in df_b.columns]
    df_t.columns = [f"t_{col}" for col in df_t.columns]
    df_final = pd.concat([df.drop(['b', 't'], axis=1), df_b, df_t], axis=1)
    # Melt the dataframe for betas
    df_melted_b = df_final.melt(id_vars=['ind', 'predictor', 'r'], 
                          value_vars=[col for col in df_final.columns if col.startswith('b_')],
                          var_name='b_type', value_name='b')
    df_melted_b_filtered = df_melted_b[df_melted_b['b_type'] == 'b_' + df_melted_b['predictor']]
    df_melted_b_filtered = df_melted_b_filtered[['ind', 'predictor', 'r', 'b']]
    # Melt the dataframe for t-values
    df_melted_t = df_final.melt(id_vars=['ind', 'predictor', 'r'], 
                                value_vars=[col for col in df_final.columns if col.startswith('t_')],
                                var_name='t_type', value_name='t')

    df_melted_t_filtered = df_melted_t[df_melted_t['t_type'] == 't_' + df_melted_t['predictor']]
    df_melted_t_filtered = df_melted_t_filtered[['ind', 'predictor', 't']]
    # Merge the two melted dataframes on 'ind' and 'predictor'
    df_combined = pd.merge(df_melted_b_filtered, df_melted_t_filtered, on=['ind', 'predictor'])
    df_combined['f'] = df['f'].values
    df_combined['p'] = df['p'].values
    df_combined = df_combined.set_index(['ind', 'predictor'])
    return df_combined

In [421]:
def run_regression(data, X, y, cluster):
    total = []
    for ind, df in data.groupby('ind'):
        results = []
        for predictor in X:
            # OLS
            X_val = df[[predictor]]
            y_val = df[[y]]
            X_val = sm.add_constant(X_val)

            # Fit model with robust standard errors clustered by calendar year
            model_year_clustered = sm.OLS(y_val, X_val).fit(cov_type='cluster', cov_kwds={'groups': df[cluster]})

            # Metrics
            results.append({
                'ind': ind,
                'predictor': predictor,
                'b': model_year_clustered.params,
                't': model_year_clustered.tvalues,
                'r': model_year_clustered.rsquared,
                'f': model_year_clustered.fvalue,
                'p': model_year_clustered.f_pvalue
            })
        total.append(set_correct_format(results))
    total = pd.concat(total, axis=0)
    return total

### Regression

In [422]:
two_regression = pd.concat([get_post_bubble_data(regression, date, ind) for date, ind in bubble_date.index.tolist()])
two_regression['Year'] = two_regression.index.get_level_values('date').year

In [423]:
characteristics = ['vol_vw', 'vol_vw_change', 'turnover_vw', 'turnover_vw_change', 'age_market_rank', 'age_tilt', 'issuance', 'btm_vwr', 'revenue_market_rank', 'cape', 'acc']

In [424]:
raw_regress = run_regression(two_regression, characteristics, 'two_vwr_ind', 'Year')
market_regress = run_regression(two_regression, characteristics, 'market_two_vwr_ind', 'Year')
rf_regress = run_regression(two_regression, characteristics, 'rf_two_vwr_ind', 'Year')

### Table

In [425]:
def collect_stats(formater, data, col_name):
    table_format = formater
    collect_r = []
    collect_b = [] 
    collect_t = []
    collect_f = []
    collect_p = []
    for _, df in data.groupby('predictor'):
        collect_r.append(df.r.mean())
        collect_b.append(df.b.mean())
        collect_t.append(df.t.mean())
        collect_f.append(df.f.mean())
        collect_p.append(df.p.mean())
    table_format[f'{col_name} b'] = collect_b
    table_format[f'{col_name} [t]'] = collect_t
    table_format[f'{col_name} R-squre'] = collect_r
    return table_format, np.array(collect_f).mean(), np.array(collect_p).mean()

In [426]:
format_table6 = pd.DataFrame(index=['Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])
format_rf_table6 = pd.DataFrame(index=['Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])
format_market_table6 = pd.DataFrame(index=['Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])

In [427]:
raw = collect_stats(format_table6, raw_regress, '24mo Raw Return')
market = collect_stats(format_market_table6, market_regress, '24mo Net of Market Return')
rf = collect_stats(format_rf_table6, rf_regress, '24mo Net of Risk-Free Return')

In [428]:
table6 = pd.concat([raw[0], rf[0], market[0]], axis=1)

In [429]:
table6.loc['Joint F-stat'] = [np.nan, raw[1], np.nan, np.nan, rf[1], np.nan, np.nan, market[1], np.nan]

In [430]:
table6.loc['p-value (Prob>F)'] = [np.nan, raw[2], np.nan, np.nan, rf[2], np.nan, np.nan, market[2], np.nan]

In [431]:
table6

,24mo Raw Return b,24mo Raw Return [t],24mo Raw Return R-squre,24mo Net of Risk-Free Return b,24mo Net of Risk-Free Return [t],24mo Net of Risk-Free Return R-squre,24mo Net of Market Return b,24mo Net of Market Return [t],24mo Net of Market Return R-squre
Volatility (VW),-0.103312,0.596260,0.196474,-0.101342,0.054068,0.195106,-0.119827,-0.414202,0.184523
Volatility (VW)- 1yr-Δ,-0.201098,0.179232,0.123896,-0.137598,0.326902,0.122033,-0.205750,0.234766,0.119763
Turnover (VW),-0.115808,-0.277043,0.033371,-0.141245,-0.480443,0.034097,0.344415,1.313369,0.081559
Turnover (VW)- 1yr-Δ,-5.871932,-0.080305,0.154782,-5.673904,-0.083671,0.154218,-6.492529,-0.040475,0.149865
Firm Age (VW),-0.025852,0.077261,0.172480,-0.025719,0.072910,0.174742,-0.028581,-0.036519,0.171717
Age tilt,0.735998,0.241237,0.070178,0.602944,0.263706,0.071335,0.932078,0.350379,0.073964
% Issuers,-34.527619,2.324183,0.166573,-28.936120,1.462805,0.162907,-61.969386,0.688232,0.164499
Book to Market (VW),4.834279,1.601774,0.102323,4.585709,1.596660,0.096639,4.418338,0.901321,0.098926
Sales Growth,-0.010108,1.301764,0.066097,-0.015034,0.822713,0.063114,-0.011039,0.507633,0.066913
CAPE,-0.514543,0.262101,0.057732,-0.816130,0.184082,0.056112,0.449751,0.789154,0.066926


# Creating Table 4 (Bubbles for Fama)

In [432]:
all_bubble = regression[regression.index.isin(bubble_date.index)]

In [433]:
table4 = pd.DataFrame(index=['Past 2-year Return', 'Excess Past 2-year Return', 'Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                           'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                           'CAPE', 'Acceleration'])

In [434]:
column_order = ['two_vwr_ind', 'rf_two_vwr_ind', 'vol_vw', 'vol_vw_change', 'turnover_vw', 
                'turnover_vw_change', 'age_market_rank', 'age_tilt', 'issuance', 'btm_vwr', 
                'revenue_market_rank', 'cape', 'acc']

In [435]:
all_ind_mean = regression[column_order].mean().values
all_ind_std = regression[column_order].std().values
table4['All industry-years Mean'] = all_ind_mean
table4['All industry-years STD'] = all_ind_std

In [436]:
run_up_mean = np.array(all_bubble[column_order].mean())
run_up_std = np.array(all_bubble[column_order].std())
table4['Run-ups Mean'] = run_up_mean
table4['Run-ups STD'] = run_up_std

### Detecting crashes

In [437]:
# Function to check if a crash occurred
def did_crash_occur(data, run_up_date):
    two_year_end = run_up_date + pd.DateOffset(years=2)
    post_runup_data = data.loc[run_up_date:two_year_end]
    
    # Calculate cumulative return from run-up date
    post_runup_data['cumulative_return'] = (1 + post_runup_data['vwr_ind']).cumprod() - 1

    # Check if there was any point where the drawdown from the max was 40% or more
    if any(post_runup_data['cumulative_return'] <= (post_runup_data['cumulative_return'].cummax() - 0.4)):
        return 1  # Crash occurred
    return 0  # No crash

In [438]:
run_up_check_crash = pd.DataFrame(columns=['date', 'ind', 'crash'])

for (run_up_date, industry) in bubble_date.index:
    crash = did_crash_occur(regression.xs(industry, level=1), run_up_date)
    run_up_check_crash = pd.concat([run_up_check_crash, pd.DataFrame({'date': [run_up_date], 'ind': [industry], 'crash': [crash]})], ignore_index=True)
    
run_up_check_crash = run_up_check_crash.set_index(['date', 'ind'])
no_crash = run_up_check_crash.loc[run_up_check_crash.crash==0]
crash = run_up_check_crash.loc[run_up_check_crash.crash==1]

In [439]:
crash_bubble = regression[regression.index.isin(crash.index)]
run_up_crash_mean = np.array(crash_bubble[column_order].mean())
run_up_crash_std = np.array(crash_bubble[column_order].std())
table4['Run-ups with Crash Mean'] = run_up_crash_mean
table4['Run-ups with Crash STD'] = run_up_crash_std

In [440]:
no_crash_bubble = regression[regression.index.isin(no_crash.index)]
run_up_no_crash_mean = np.array(no_crash_bubble[column_order].mean())
run_up_no_crash_std = np.array(no_crash_bubble[column_order].std())
table4['Run-ups with no Crash Mean'] = run_up_no_crash_mean
table4['Run-ups with no Crash STD'] = run_up_no_crash_std

In [441]:
table4['Crash minus no Crash Mean'] = table4['Run-ups with Crash Mean'] - table4['Run-ups with no Crash Mean']

### Regression

In [442]:
def t_stat(data, X, y, cluster):
    results = []
    X_val = data[X]
    y_val = data[[y]]
    X_val = sm.add_constant(X_val)
    # Fit model with robust standard errors clustered by calendar year
    model_year_clustered = sm.OLS(y_val, X_val).fit(cov_type='cluster', cov_kwds={'groups': data[cluster]})

    # Metrics
    results.append({
        'b': model_year_clustered.params,
        't': model_year_clustered.tvalues,
        'r': model_year_clustered.rsquared,
        'f': model_year_clustered.fvalue,
        'p': model_year_clustered.f_pvalue
    })
    
    # 2. Extract series data into separate columns for 'b'
    df = pd.DataFrame(results)
    # Extract the coefficients ('b') and t-values ('t')
    # df_b = df['b'].apply(pd.Series)
    df_t = df['t'].apply(pd.Series)
    # Rename columns to indicate they contain coefficients or t-values
    # df_b.columns = [f"b_{col}" for col in df_b.columns]
    df_t.columns = [f"{col}" for col in df_t.columns]
    # df_final = pd.concat([df.drop(['b', 't'], axis=1), df_b, df_t], axis=1)
    df_final = pd.concat([df.drop(['b', 'r', 'f', 'p', 't'], axis=1), df_t.drop('const', axis=1)], axis=1)
    return df_final

In [443]:
characteristics = ['two_vwr_ind', 'rf_two_vwr_ind', 'vol_vw', 'vol_vw_change', 'turnover_vw', 'turnover_vw_change', 'age_market_rank', 'age_tilt', 'issuance', 'btm_vwr', 'revenue_market_rank', 'cape', 'acc']

In [444]:
no_crash_regression = pd.concat([get_post_bubble_data(regression, date, ind) for date, ind in no_crash.index.tolist()])
no_crash_regression['Year'] = no_crash_regression.index.get_level_values('date').year
no_crash_regression['Ind'] = no_crash_regression.index.get_level_values('ind')
no_crash_regression['Year_Ind'] = no_crash_regression['Year'].astype(str) + '_' + no_crash_regression['Ind'].astype(str)

In [445]:
crash_regression = pd.concat([get_post_bubble_data(regression, date, ind) for date, ind in crash.index.tolist()])
crash_regression['Year'] = crash_regression.index.get_level_values('date').year
crash_regression['Ind'] = crash_regression.index.get_level_values('ind')
crash_regression['Year_Ind'] = crash_regression['Year'].astype(str) + '_' + crash_regression['Ind'].astype(str)

In [446]:
no_crash_stats = run_regression(no_crash_regression, characteristics, 'vwr_ind', 'Year_Ind')
crash_stats = run_regression(crash_regression, characteristics, 'vwr_ind', 'Year_Ind')

In [447]:
format_table4_no_crash = pd.DataFrame(index=['Past 2-year Return', 'Excess Past 2-year Return', 'Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])
format_table4_crash = pd.DataFrame(index=['Past 2-year Return', 'Excess Past 2-year Return', 'Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])

In [448]:
no_crash_output = collect_stats(format_table4_no_crash, no_crash_stats, 'No Crash')
crash_output = collect_stats(format_table4_crash, crash_stats, 'Crash')

In [449]:
table4['Crash minus no Crash [t]'] = crash_output[0]['Crash [t]'] - no_crash_output[0]['No Crash [t]']

### SUR

In [450]:
def get_stats(data):
    sur_columns = ['two_vwr_ind', 'rf_two_vwr_ind', 'vol_vw', 'vol_vw_change', 'turnover_vw', 
                   'turnover_vw_change', 'age_market_rank', 'issuance', 'age_tilt', 'btm_vwr', 
                   'revenue_market_rank', 'cape', 'acc']
    f = []
    p = []
    for col in sur_columns:
        y = data['vwr_ind']
        X = sm.add_constant(data[col].astype(float))
        model = sm.OLS(y, X).fit()
        f.append(model.fvalue)
        p.append(model.f_pvalue)
    return np.nanmean(np.array(f)), np.nanmean(np.array(p))
    
def run_sur_all(data):
    f_total = []
    p_total = []
    for ind, df in data.groupby('ind'):
        test = get_stats(df)
        f_total.append(test[0])
        p_total.append(test[1])
    return np.nanmean(np.array(f_total)), np.nanmean(np.array(p_total))

In [451]:
# crash_sur_stats = run_sur_all(crash_regression)
# no_crash_sur_stats = run_sur_all(crash_regression)
joint_stats = run_sur_all(regression)
f = joint_stats[0]
p = joint_stats[1]

In [452]:
table4.loc['Joint F-stat'] = [np.nan] * (table4.shape[1]-1) + [f]

In [453]:
table4.loc['p-value (Prob>F)'] = [np.nan] * (table4.shape[1]-1) + [p]

In [454]:
table4

,All industry-years Mean,All industry-years STD,Run-ups Mean,Run-ups STD,Run-ups with Crash Mean,Run-ups with Crash STD,Run-ups with no Crash Mean,Run-ups with no Crash STD,Crash minus no Crash Mean,Crash minus no Crash [t]
Past 2-year Return,0.444377,0.509573,0.341007,0.383846,0.329000,0.347725,0.355681,0.433859,-0.026681,3.418650
Excess Past 2-year Return,0.421324,0.510784,0.303780,0.382116,0.279225,0.348582,0.333791,0.427944,-0.054567,1.733678
Volatility (VW),0.021038,0.061303,0.027811,0.013560,0.032630,0.016649,0.021922,0.003558,0.010708,-2.721270
Volatility (VW)- 1yr-Δ,0.113270,2.634499,0.396626,1.287813,0.685217,1.662391,0.043904,0.389110,0.641314,0.034077
Turnover (VW),0.062541,0.085726,0.108521,0.137164,0.138228,0.171267,0.072212,0.066250,0.066016,-0.369589
Turnover (VW)- 1yr-Δ,0.221933,2.090064,0.476511,0.790627,0.585179,0.944082,0.343694,0.546870,0.241485,-1.766230
Firm Age (VW),0.520179,0.319162,0.337691,0.273505,0.346817,0.288113,0.326536,0.262356,0.020281,-1.777679
Age tilt,-0.013479,0.090710,-0.102511,0.082055,-0.105966,0.096413,-0.098289,0.062687,-0.007676,1.096314
% Issuers,0.003025,0.012900,0.010866,0.026017,0.016063,0.032394,0.004514,0.013376,0.011549,-0.932365
Book to Market (VW),0.999023,5.123299,0.579723,0.919324,0.792711,1.183920,0.319404,0.277420,0.473308,-1.889017
